# Smart Data Cleaner & Analyzer

In [2]:
from functools import reduce
data = {
    "Region1": [
        {"name": "Ibad ", "score": 88},
        {"name": "Izaz", "score": -10},  # invalid
        {"name": "Salif", "score": 70}
    ],
    "Region2": [
        {"name": " Shuhood", "score": 95},
        {"name": "Huzaifa", "score": 58}
    ]
}

def clean_record(record):
    record['name'] = record['name'].strip().title()
    assert record['score'] >= 0 , f"Invilid score for {record['name']}"
    return record

def clean_all(data,cleaner):
    cleaned = {}
    for region, students in data.items():
        cleaned_students = []
        for student in students:
            try:
                cleaned_students.append(cleaner(student))
            except AssertionError as e:
                print("Data Error:", e)
        cleaned[region] = cleaned_students
    return cleaned

def is_passing(student):
    return student['score'] >=60

def filter_passing(data):
    return {
        region: list(filter(is_passing, students))
        for region,students in data.items()
    }

def average_score(students):
    if not students:
        return 0
    total = reduce(lambda acc,s: acc + s["score"],students,0)
    return total/len(students)

def region_averages(data):
    return {
        region: average_score(students)
        for region,students in data.items()
    }

def main():
    cleaned = clean_all(data, clean_record)
    print("\nCleaned Data:", cleaned)

    passed = filter_passing(cleaned)
    print("\nPassed Students:", passed)

    averages = region_averages(cleaned)
    print("\nRegion Averages:", averages)

main()


Data Error: Invilid score for Izaz

Cleaned Data: {'Region1': [{'name': 'Ibad', 'score': 88}, {'name': 'Salif', 'score': 70}], 'Region2': [{'name': 'Shuhood', 'score': 95}, {'name': 'Huzaifa', 'score': 58}]}

Passed Students: {'Region1': [{'name': 'Ibad', 'score': 88}, {'name': 'Salif', 'score': 70}], 'Region2': [{'name': 'Shuhood', 'score': 95}]}

Region Averages: {'Region1': 79.0, 'Region2': 76.5}


# Smart Student Analyzer

In [62]:
from functools import reduce

# --- Step 1: Raw Input ---
students = [" Ali", "SARA", "bilal ", "hina"]
scores = [88, 92, -10, 54]
subjects = ["Math", "Math", "Math", "Math"]
attendance = [0.95, 0.87, 0.20, 0.60]

# --- Step 2: Data Cleaner ---
def clean_record(name, score, subject, attendance):
    name = name.strip().title()
    assert 0 <= score <= 100, f"Invalid score for {name}"
    assert 0 <= attendance <= 1, f"Invalid attendance for {name}"
    return {"name": name, "score": score, "subject": subject, "attendance": attendance}

records = []

for i, (n, s, sub, att) in enumerate(zip(students, scores, subjects, attendance)):
    try:
        record = clean_record(n, s, sub, att)
        records.append(record)
    except AssertionError as e:
        print("Validation Error:", e)

# --- Step 3: Generator for Pass/Fail ---
def evaluate_pass_fail(score_list):
    for score in score_list:
        yield "Pass" if score >= 60 else "Fail"

# Attach result to records
pass_fail_gen = evaluate_pass_fail([r["score"] for r in records])
for r, result in zip(records, pass_fail_gen):
    r["result"] = result

# --- Step 4: Dictionary from Records (List Comprehension + Zip) ---
score_dict = {r["name"]: r["score"] for r in records}

# --- Step 5: Filtering Passed Students ---
passed = list(filter(lambda r: r["result"] == "Pass", records))

# --- Step 6: Reduce to Find Average Score ---
def average(scores):
    if not scores:
        return 0
    return reduce(lambda a, b: a + b, scores) / len(scores)

avg_score = average([r["score"] for r in passed])

# --- Step 7: Display with Enumerate ---
print("\n Final Report:")
for i, r in enumerate(records, start=1):
    print(f"{i}. {r['name']} - {r['score']} in {r['subject']} - {r['result']}")

print(f"\n Average Score (Passed): {avg_score:.2f}")

# --- Step 8: Using Iterators Manually ---
print("\n Manual iteration over passed students:")
it = iter(passed)
while True:
    try:
        student = next(it)
        print(f"{student['name']} passed with {student['score']} points")
    except StopIteration:
        break


Validation Error: Invalid score for Bilal

 Final Report:
1. Ali - 88 in Math - Pass
2. Sara - 92 in Math - Pass
3. Hina - 54 in Math - Fail

 Average Score (Passed): 90.00

 Manual iteration over passed students:
Ali passed with 88 points
Sara passed with 92 points


# Customer Churn Risk Analyzer

In [42]:
from functools import reduce

# Raw Data
names = [" Ibad", "Izaz", "Huzaifa", "Salif"]
logins = [3, 12, 6, 1]
tickets = [2, 0, 1, 3]
plans = ["Basic", "Pro", "Basic", "Enterprise"]
usage_hours = [5, 40, 15, 2]
satisfaction = [4, 9, 6, 3]

# Step 1: Clean & Validate
def clean_customers(name, logins, tickets, plan, usage, score):
    name = name.strip().title()
    assert 0 <= logins <= 31, "Invalid login days"
    assert usage >= 0, "Negative usage"
    assert 0 <= score <= 10, "Satisfaction score out of range"
    return {
        "name": name,
        "logins": logins,
        "tickets": tickets,
        "plan": plan,
        "usage": usage,
        "score": score
    }

customers = []
for data in zip(names, logins, tickets, plans, usage_hours, satisfaction):
    try:
        customers.append(clean_customers(*data))
    except AssertionError as e:
        print("Assertion Error:", e)

# Step 2: Generator to Tag Churn Risk
def risk_generator(customers):
    for c in customers:
        if c['logins'] <= 3 or c['score'] <= 4:
            yield "High Risk"
        elif c['logins'] <= 6 or c['score'] <= 6:
            yield "Medium Risk"
        else:
            yield "Low Risk"

# Step 3: Assign Risk from Generator
risk_gen = risk_generator(customers)
for c, risk in zip(customers, risk_gen):
    c['risk'] = risk

# Step 4: Tag Discounts Based on Risk
for c in customers:
    if c['risk'] == "High Risk":
        c['discount'] = "30%"
    elif c['risk'] == "Medium Risk":
        c['discount'] = "15%"
    else:
        c['discount'] = "5%"

# Step 5: Filter High Risk Customers
high_risk = list(filter(lambda c: c["risk"] == "High Risk", customers))

# Step 6: Aggregations with Reduce
total_usage = reduce(lambda a, c: a + c["usage"], customers, 0)
avg_score = reduce(lambda a, c: a + c["score"], customers, 0) / len(customers)

# Report
print("\n Churn Risk Report")
for i, c in enumerate(customers, 1):
    print(f"{i}. {c['name']} | Logins: {c['logins']} | Risk: {c['risk']} | Discount: {c['discount']}")

# Contact List
print("\n Customers to Contact for Retention:")
it = iter(high_risk)
while True:
    try:
        customer = next(it)
        print(f" {customer['name']} - Plan: {customer['plan']} - Score: {customer['score']}")
    except StopIteration:
        break

#  Summary
print(f"\n Total Usage Hours: {total_usage}")
print(f" Average Satisfaction Score: {avg_score:.2f}")



 Churn Risk Report
1. Ibad | Logins: 3 | Risk: High Risk | Discount: 30%
2. Izaz | Logins: 12 | Risk: Low Risk | Discount: 5%
3. Huzaifa | Logins: 6 | Risk: Medium Risk | Discount: 15%
4. Salif | Logins: 1 | Risk: High Risk | Discount: 30%

 Customers to Contact for Retention:
 Ibad - Plan: Basic - Score: 4
 Salif - Plan: Enterprise - Score: 3

 Total Usage Hours: 62
 Average Satisfaction Score: 5.50
